In [54]:
import json
import os
from collections import OrderedDict
dir = "avey-results/100"
def load(filepath: str):
    with open(f"{dir}/{filepath}", "r") as file:
        data = json.load(file)
    try:
        return data["Final"]["diseases"]
    except Exception as e:
        print(filepath)
        raise e

files = [f for f in os.listdir(dir) if os.path.isfile(os.path.join(dir,f))]
cases = OrderedDict()
for f in files:
    cases[f.split(".")[0]] = load(f)
list(cases.items())[0]


('10',
 [{'disease': 'acute appendicitis',
   'instance': 'acute appendicitis #1',
   'confidence': 11068328.49694255,
   'confidence_age': 284020.8239472373,
   'probability': 1.0064098245810674e-07},
  {'disease': 'diverticulitis',
   'instance': 'diverticulitis #1',
   'confidence': 36.338529287898076,
   'confidence_age': 0.14088451396072943,
   'probability': 6.7655871484687075e-15},
  {'disease': 'acute mesenteric ischemia',
   'instance': 'acute mesenteric ischemia #1',
   'confidence': 219.36832691031228,
   'confidence_age': 0.410916468744483,
   'probability': 3.907518149189293e-15},
  {'disease': 'irritable bowel syndrome',
   'instance': 'irritable bowel syndrome #1',
   'confidence': 0.03538302629837439,
   'confidence_age': 0.00042575701715706925,
   'probability': 1.4231512543831422e-15},
  {'disease': 'acute cholecystitis',
   'instance': 'acute cholecystitis #1',
   'confidence': 4.627164247073902,
   'confidence_age': 0.03988438718779551,
   'probability': 1.216647560

In [55]:
class Disease():
    def __init__(self,dictionary):
        self.name = dictionary["disease"]
        self.confidenceAge = dictionary["confidence_age"]
        self.confidence = dictionary["confidence"]
        self.probability = dictionary["probability"]

    # def __lt__(self,other):
    #     return self.confidenceAge < other.confidenceAge

    def __str__(self):
        return self.name + f"  |  P {self.probability}"+f"  |  C {self.confidence}"

caseObjects = OrderedDict()
for id, case in cases.items():
    try:
        caseObjects[id] = [Disease(d) for d in case]
    except Exception as e:
        print(id)
        # raise e
    
list(caseObjects.items())[0]


51
19
20
9
46
2
24
27
14
60
7
58
168
82
18
63


('10',
  <__main__.Disease at 0x7fcbc81c9340>])

In [56]:
def sort(data, key):
    return sorted(data, key=key, reverse=True)


def filter(l, func):
    return [d for d in l if func(d)]



In [57]:
# sorts by probability and takes the top 6
def applyRule1():
    return {
        id: filter(
            sort(caseObject, lambda x: x.probability),
            lambda x: x.confidence >= 1
        )[:6] for id, caseObject in caseObjects.items()
    }

# sorts by confidenceAge and takes the top 6


def applyRule2():
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.confidence >= 1
        )[:6] for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability
def applyRule3():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id]
        )[:6] for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability and top 6 by confidence
def applyRule4():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    cs = {
        id: sort(caseObject, key=lambda x: x.confidence)[:6][-1].confidence
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id] or x.confidence >= cs[id]
        )[:6] for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability or top 6 by confidence
def applyRule5():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    cs = {
        id: sort(caseObject, key=lambda x: x.confidence)[:6][-1].confidence
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id] and x.confidence >= cs[id]
        )[:6] for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability or top 6 by confidence
def applyRule6():
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.confidence >= 1
        ) for id, caseObject in caseObjects.items()
    }


results = applyRule6()
list({id: [x.name for x in l] for id, l in results.items()}.items())[0]


('10',
 ['acute appendicitis',
  'acute mesenteric ischemia',
  'diverticulitis',
  'acute cholecystitis',
  'acute cholangitis'])

In [58]:
import json
label = "SortConfidenceAgeConfMoreThan1"
with open(f"experiments/{label}.json", "w") as file:
    json.dump(
        [
            {"case_number" : id,
             "app": label,
                "content": "; ".join([x.name for x in l])
            }
            for id, l in results.items()
        ], file, indent=4,
    )


In [59]:
# import matplotlib.pyplot as plt

# id = 50
# points = [d.confidence for d in caseObjects[id]]
# plt.scatter(range(1,len(points)+1), sorted(points))
# plt.rcParams.update({'figure.figsize': (10, 8), 'figure.dpi': 100})
# plt.title(f'Case {id}')
# plt.xlabel('point number')
# plt.ylabel('confidence')
# plt.show()